In [1]:
import pandas as pd
import locale
import pickle
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [2]:
with open('../src/depara/trends/categoria.pkl', 'rb') as arq:
    lista_categoria = pickle.load(arq)
lista_categoria

,ID,NOME_CATEGORIA
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [3]:
dtypes = {
    'ID_CATEGORIA' : 'int32',
    'ID_CANAL' : 'string',
    'NM_CANAL' : 'string',
    'ID_VIDEO' : 'string',
    'TITULO_VIDEO' : 'string',
    'DURACAO' : 'string',
    'DESCRICAO': 'string',
    'TOTAL_COMENTARIOS' : 'float32',
    'TOTAL_FAVORITOS': 'int32',
    'TOTAL_LIKES': 'float32',
    'TOTAL_VISUALIZACOES': 'int32'
}

# Funções gerais

In [4]:
def calcular_variacoes_gerais(dataframe: pd.DataFrame) -> pd.DataFrame:
    dataframe['TOTAL_VISUALIZACOES_DESLOCADO'] = dataframe.groupby(['ID_VIDEO'])['TOTAL_VISUALIZACOES'].shift(1)
    dataframe.fillna(0, axis=1, inplace=True)
    dataframe['TOTAL_VISUALIZACOES_DIA_ATUAL'] = dataframe['TOTAL_VISUALIZACOES'] - dataframe['TOTAL_VISUALIZACOES_DESLOCADO']

    dataframe['TOTAL_COMENTARIOS_DESLOCADO'] = dataframe.groupby(['ID_VIDEO'])['TOTAL_COMENTARIOS'].shift(1)
    dataframe.fillna(0, axis=1, inplace=True)
    dataframe['TOTAL_COMENTARIOS_DIA_ATUAL'] = dataframe['TOTAL_COMENTARIOS'] - dataframe['TOTAL_COMENTARIOS_DESLOCADO']
    
    dataframe['TOTAL_LIKES_DESLOCADO'] = dataframe.groupby(['ID_VIDEO'])['TOTAL_LIKES'].shift(1)
    dataframe.fillna(0, axis=1, inplace=True)
    dataframe['TOTAL_LIKES_DIA_ATUAL'] = dataframe['TOTAL_LIKES'] - dataframe['TOTAL_LIKES_DESLOCADO']
    display(dataframe)
    dataframe.drop(['TOTAL_COMENTARIOS_DESLOCADO', 'TOTAL_COMENTARIOS', 'TOTAL_VISUALIZACOES_DESLOCADO', 'TOTAL_VISUALIZACOES', 'TOTAL_LIKES_DESLOCADO', 'TOTAL_LIKES'], axis=1, inplace=True)

    return dataframe

In [5]:
def traduzir_dia_da_semana(dia):
    traducao = {
        'Sunday': 'Domingo',
        'Monday': 'Segunda-feira',
        'Tuesday': 'Terça-feira',
        'Wednesday': 'Quarta-feira',
        'Thursday': 'Quinta-feira',
        'Friday': 'Sexta-feira',
        'Saturday': 'Sábado'
    }
    return traducao.get(dia, dia)

In [6]:
def turno_semana(hora: int):
    if 0 <= hora < 6:
        turno = 'Madrugada', 1
    elif 6 <= hora < 12:
        turno = 'Manhã' , 2
    elif 12 <= hora < 18:
        turno = 'Tarde', 3
    else:
        turno = 'Noite', 4
    return turno

# Abrir dataframe

In [7]:
datas = [
    'extracao_data_2023_10_15',
    'extracao_data_2023_10_16', 
    'extracao_data_2023_10_17', 
    'extracao_data_2023_10_18',
    'extracao_data_2023_10_19',
    'extracao_data_2023_10_20',
    'extracao_data_2023_10_21',
    'extracao_data_2023_10_22',
    'extracao_data_2023_10_23',
    'extracao_data_2023_10_24',
    'extracao_data_2023_10_25',
    'extracao_data_2023_10_26',
    'extracao_data_2023_10_27']
base_original = pd.DataFrame()
for path in datas:
    try:
        base_temp = pd.read_parquet(f'../data/prata/top_brazil/{path}/historico_top_brazil.parquet')
        base_original = pd.concat([base_original, base_temp])
    except Exception as e:
        print(path)
        print()
        pass

extracao_data_2023_10_17

extracao_data_2023_10_18



In [8]:
base_original.head()

,data_extracao,ID_CATEGORIA,ID_CANAL,NM_CANAL,ID_VIDEO,TITULO_VIDEO,DURACAO,DESCRICAO,TOTAL_COMENTARIOS,TOTAL_FAVORITOS,TOTAL_LIKES,TOTAL_VISUALIZACOES
0,2023-10-15 14:38:56,24,UCX6OQ3DkcsbYNE6H8uQQuVA,MrBeast,3ryID_SwU5E,"$1 Vs $100,000,000 House!",PT17M36S,I can’t believe how expensive the last house i...,100754,0,3182804,64102803
1,2023-10-15 14:38:56,25,UC376n347Ob5Lwzq2WGzF1AA,SBT News,146wNN1qwPk,Terminou prazo de Israel para que palestinos d...,PT8M41S,"Terminou, no início desta noite (13.out), o pr...",2683,0,19545,1050551
2,2023-10-15 14:38:56,20,UCmhx7Q1WRB85e7bF-MHcehA,Ligonz,uqN9GrkAxOI,FUI EXPULSO DA CREATIVE SQUAD 4,PT15M,#creativesquad4 #cs4 #expulso #genshinimpact ...,335,0,11423,239520
3,2023-10-15 14:38:56,20,UCcqcrpCct3dlF8H-3lv7sKA,Nait,RUNExoWpWbc,24 HORAS para ser DESBANIDO da Creative Squad,PT11M55S,BAIXE AGORA GENSHIN IMPACT E VENHA CONHECER ES...,220,0,8113,138405
4,2023-10-15 14:38:56,10,UCRD1ypvmK7e_05Rtu9zHOfQ,Ana Castela,IgG1ekanyFQ,Ana Castela - Alerta de Golpe (DVD Boiadeira I...,PT2M30S,Ouça agora: https://agroplaybr.ffm.to/boiadeir...,1789,0,29201,931886


In [9]:
base_original.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29399 entries, 0 to 2349
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   data_extracao        29399 non-null  object
 1   ID_CATEGORIA         29399 non-null  object
 2   ID_CANAL             29399 non-null  object
 3   NM_CANAL             29399 non-null  object
 4   ID_VIDEO             29399 non-null  object
 5   TITULO_VIDEO         29399 non-null  object
 6   DURACAO              29399 non-null  object
 7   DESCRICAO            29399 non-null  object
 8   TOTAL_COMENTARIOS    28472 non-null  object
 9   TOTAL_FAVORITOS      29399 non-null  object
 10  TOTAL_LIKES          28796 non-null  object
 11  TOTAL_VISUALIZACOES  29399 non-null  object
dtypes: object(12)
memory usage: 2.9+ MB


In [10]:
base_original['data_extracao'] = base_original['data_extracao'].astype('datetime64[ns]')
base_original['ID_CATEGORIA'] = base_original['ID_CATEGORIA'].astype('string')
base_original['ID_CANAL'] = base_original['ID_CANAL'].astype('string')
base_original['NM_CANAL'] = base_original['NM_CANAL'].astype('string')
base_original['ID_VIDEO'] = base_original['ID_VIDEO'].astype('string')
base_original['TITULO_VIDEO'] = base_original['TITULO_VIDEO'].astype('string')
base_original['DURACAO'] = base_original['DURACAO'].astype('string')
base_original['DESCRICAO'] = base_original['DESCRICAO'].astype('string')
base_original['TOTAL_COMENTARIOS'] = base_original['TOTAL_COMENTARIOS'].astype('float32')
base_original['TOTAL_FAVORITOS'] = base_original['TOTAL_FAVORITOS'].astype('float32')
base_original['TOTAL_LIKES'] = base_original['TOTAL_LIKES'].astype('float32')
base_original['TOTAL_VISUALIZACOES'] = base_original['TOTAL_VISUALIZACOES'].astype('float32')


In [11]:
base_original['TOTAL_FAVORITOS'] = base_original['TOTAL_FAVORITOS'].astype('int32',errors='ignore')
base_original['TOTAL_LIKES'] = base_original['TOTAL_LIKES'].astype('int32', errors='ignore')
base_original['TOTAL_VISUALIZACOES'] = base_original['TOTAL_VISUALIZACOES'].astype('int32', errors='ignore')


In [12]:
base_original.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29399 entries, 0 to 2349
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   data_extracao        29399 non-null  datetime64[ns]
 1   ID_CATEGORIA         29399 non-null  string        
 2   ID_CANAL             29399 non-null  string        
 3   NM_CANAL             29399 non-null  string        
 4   ID_VIDEO             29399 non-null  string        
 5   TITULO_VIDEO         29399 non-null  string        
 6   DURACAO              29399 non-null  string        
 7   DESCRICAO            29399 non-null  string        
 8   TOTAL_COMENTARIOS    28472 non-null  float32       
 9   TOTAL_FAVORITOS      29399 non-null  int32         
 10  TOTAL_LIKES          28796 non-null  float32       
 11  TOTAL_VISUALIZACOES  29399 non-null  int32         
dtypes: datetime64[ns](1), float32(2), int32(2), string(7)
memory usage: 2.5 MB


In [13]:
base_original['DATA_EXTRACAO_API'] = base_original['data_extracao']
base_original.drop('data_extracao', axis=1, inplace=True)

In [14]:
base_original['DATA_EXTRACAO'] = base_original['DATA_EXTRACAO_API'].dt.date.astype('datetime64[ns]')
base_original['HORA_EXTRACAO'] = base_original['DATA_EXTRACAO_API'].dt.hour
base_original['TURNO_EXTRACAO'] = base_original['HORA_EXTRACAO'].apply(lambda x : turno_semana(x)[0]).astype('string')
base_original['SEMANA_EXTRACAO'] = base_original['DATA_EXTRACAO_API'].dt.strftime('%A').str.capitalize().astype('string')
base_original['INDICE_TURNO_EXTRACAO'] =  base_original['HORA_EXTRACAO'].apply(lambda x : turno_semana(x)[1]).astype('int32')
base_original['INDICE_SEMANA_EXTRACAO'] = base_original['DATA_EXTRACAO'].dt.day_of_week
base_original

,ID_CATEGORIA,ID_CANAL,NM_CANAL,ID_VIDEO,TITULO_VIDEO,DURACAO,DESCRICAO,TOTAL_COMENTARIOS,TOTAL_FAVORITOS,TOTAL_LIKES,TOTAL_VISUALIZACOES,DATA_EXTRACAO_API,DATA_EXTRACAO,HORA_EXTRACAO,TURNO_EXTRACAO,SEMANA_EXTRACAO,INDICE_TURNO_EXTRACAO,INDICE_SEMANA_EXTRACAO
0,24,UCX6OQ3DkcsbYNE6H8uQQuVA,MrBeast,3ryID_SwU5E,"$1 Vs $100,000,000 House!",PT17M36S,I can’t believe how expensive the last house i...,100754.0,0,3182804.0,64102804,2023-10-15 14:38:56,2023-10-15,14,Tarde,Domingo,3,6
1,25,UC376n347Ob5Lwzq2WGzF1AA,SBT News,146wNN1qwPk,Terminou prazo de Israel para que palestinos d...,PT8M41S,"Terminou, no início desta noite (13.out), o pr...",2683.0,0,19545.0,1050551,2023-10-15 14:38:56,2023-10-15,14,Tarde,Domingo,3,6
2,20,UCmhx7Q1WRB85e7bF-MHcehA,Ligonz,uqN9GrkAxOI,FUI EXPULSO DA CREATIVE SQUAD 4,PT15M,#creativesquad4 #cs4 #expulso #genshinimpact ...,335.0,0,11423.0,239520,2023-10-15 14:38:56,2023-10-15,14,Tarde,Domingo,3,6
3,20,UCcqcrpCct3dlF8H-3lv7sKA,Nait,RUNExoWpWbc,24 HORAS para ser DESBANIDO da Creative Squad,PT11M55S,BAIXE AGORA GENSHIN IMPACT E VENHA CONHECER ES...,220.0,0,8113.0,138405,2023-10-15 14:38:56,2023-10-15,14,Tarde,Domingo,3,6
4,10,UCRD1ypvmK7e_05Rtu9zHOfQ,Ana Castela,IgG1ekanyFQ,Ana Castela - Alerta de Golpe (DVD Boiadeira I...,PT2M30S,Ouça agora: https://agroplaybr.ffm.to/boiadeir...,1789.0,0,29201.0,931886,2023-10-15 14:38:56,2023-10-15,14,Tarde,Domingo,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,20,UCKAdZfnAhMY9RFSD4qL4KCQ,Tigre TV,EV8_D0Mx6aI,O DIA CHEGOU!!! ATUALIZAÇÃO 20 do BLOX FRUITS ...,PT12M43S,Use StarCode ⇢ TigreTV ⇠ e ajude o tigrinho he...,502.0,0,4833.0,101571,2023-10-27 19:02:33,2023-10-27,19,Noite,Sexta-feira,4,4
2346,10,UCDVAzIDnbgElNB4XCrYLvsA,Bruno Gadiol,fDbNLQSddTQ,"Bruno Gadiol, Grag Queen - f:)da-se (Videoclip...",PT2M55S,"Esse é o videoclipe oficial de ""f:)da-se"". Cl...",362.0,0,7590.0,62678,2023-10-27 19:02:33,2023-10-27,19,Noite,Sexta-feira,4,4
2347,10,UCfUEYnV7UjMRfBGY0bfEkPA,JOTAPÊ,Ol_t-hgXrv0,JOTAPÊ - JARDIM DAS FLORES (Prod. Marquinho no...,PT3M23S,Ouça #jardimdasflores em todas as plataformas ...,4034.0,0,66104.0,538508,2023-10-27 19:02:33,2023-10-27,19,Noite,Sexta-feira,4,4
2348,10,UCFZsvdD_Sb98qFdtSU7Et2Q,Tz da Coronel,XyA8jDHTEcU,Tz da Coronel - Castelo de Areia feat. Veigh &...,PT3M43S,Ouça o álbum #DACOROMODE em todas as plataform...,1271.0,0,46277.0,1003517,2023-10-27 19:02:33,2023-10-27,19,Noite,Sexta-feira,4,4


# popularidade categoria

- Valor max por turno

In [15]:
base_original.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29399 entries, 0 to 2349
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID_CATEGORIA            29399 non-null  string        
 1   ID_CANAL                29399 non-null  string        
 2   NM_CANAL                29399 non-null  string        
 3   ID_VIDEO                29399 non-null  string        
 4   TITULO_VIDEO            29399 non-null  string        
 5   DURACAO                 29399 non-null  string        
 6   DESCRICAO               29399 non-null  string        
 7   TOTAL_COMENTARIOS       28472 non-null  float32       
 8   TOTAL_FAVORITOS         29399 non-null  int32         
 9   TOTAL_LIKES             28796 non-null  float32       
 10  TOTAL_VISUALIZACOES     29399 non-null  int32         
 11  DATA_EXTRACAO_API       29399 non-null  datetime64[ns]
 12  DATA_EXTRACAO           29399 non-null  datetime64[n

In [16]:
base_original['DATA_EXTRACAO'].unique()

<DatetimeArray>
['2023-10-15 00:00:00', '2023-10-16 00:00:00', '2023-10-19 00:00:00',
 '2023-10-20 00:00:00', '2023-10-21 00:00:00', '2023-10-22 00:00:00',
 '2023-10-23 00:00:00', '2023-10-24 00:00:00', '2023-10-25 00:00:00',
 '2023-10-26 00:00:00', '2023-10-27 00:00:00']
Length: 11, dtype: datetime64[ns]

In [17]:
base_original = base_original.sort_values(by=['DATA_EXTRACAO', 'INDICE_TURNO_EXTRACAO', 'ID_CATEGORIA', 'ID_CANAL', 'ID_VIDEO'])


In [18]:
base_max = base_original[['DATA_EXTRACAO', 'TURNO_EXTRACAO','INDICE_TURNO_EXTRACAO', 'ID_CATEGORIA', 'ID_CANAL', 'ID_VIDEO', 'TOTAL_VISUALIZACOES']]
base_max.query('ID_VIDEO == "8KVsaoveTbw" and DATA_EXTRACAO == "2023-10-15" ').sort_values(by=['DATA_EXTRACAO', 'INDICE_TURNO_EXTRACAO'])

,DATA_EXTRACAO,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,ID_CATEGORIA,ID_CANAL,ID_VIDEO,TOTAL_VISUALIZACOES
64,2023-10-15,Tarde,3,1,UCuPivVjnfNo4mb3Oog_frZg,8KVsaoveTbw,4029760
264,2023-10-15,Tarde,3,1,UCuPivVjnfNo4mb3Oog_frZg,8KVsaoveTbw,4030703
464,2023-10-15,Tarde,3,1,UCuPivVjnfNo4mb3Oog_frZg,8KVsaoveTbw,4033786
664,2023-10-15,Tarde,3,1,UCuPivVjnfNo4mb3Oog_frZg,8KVsaoveTbw,4042055
864,2023-10-15,Tarde,3,1,UCuPivVjnfNo4mb3Oog_frZg,8KVsaoveTbw,4042055
1064,2023-10-15,Tarde,3,1,UCuPivVjnfNo4mb3Oog_frZg,8KVsaoveTbw,4058589
1264,2023-10-15,Tarde,3,1,UCuPivVjnfNo4mb3Oog_frZg,8KVsaoveTbw,4076406
1463,2023-10-15,Tarde,3,1,UCuPivVjnfNo4mb3Oog_frZg,8KVsaoveTbw,4094821
1695,2023-10-15,Tarde,3,1,UCuPivVjnfNo4mb3Oog_frZg,8KVsaoveTbw,4145625
1894,2023-10-15,Noite,4,1,UCuPivVjnfNo4mb3Oog_frZg,8KVsaoveTbw,4164634


In [19]:
base_max = base_original.groupby(['DATA_EXTRACAO' ,'ID_CATEGORIA', 'ID_VIDEO', 'TITULO_VIDEO']).agg(
    TOTAL_VISUALIZACOES=('TOTAL_VISUALIZACOES', 'max'),
    TOTAL_COMENTARIOS=('TOTAL_COMENTARIOS', 'max'),
    TOTAL_LIKES=('TOTAL_LIKES', 'max'),

).reset_index().sort_values(by=['ID_CATEGORIA', 'ID_VIDEO', 'DATA_EXTRACAO'])
base_max

,DATA_EXTRACAO,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES,TOTAL_COMENTARIOS,TOTAL_LIKES
0,2023-10-15,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,4279413,5415.0,78174.0
231,2023-10-16,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,4695234,5706.0,83626.0
471,2023-10-19,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,5407456,6045.0,89762.0
1,2023-10-15,1,CYcrmsdZuyw,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,4141018,17758.0,171952.0
232,2023-10-16,1,CYcrmsdZuyw,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,4355662,18443.0,181368.0
...,...,...,...,...,...,...,...
230,2023-10-15,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,145976,236.0,2956.0
470,2023-10-16,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,147478,233.0,2970.0
700,2023-10-19,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,151752,232.0,2984.0
921,2023-10-20,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,153609,232.0,2985.0


In [20]:
base_max = calcular_variacoes_gerais(base_max)
base_max

,DATA_EXTRACAO,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES,TOTAL_COMENTARIOS,TOTAL_LIKES,TOTAL_VISUALIZACOES_DESLOCADO,TOTAL_VISUALIZACOES_DIA_ATUAL,TOTAL_COMENTARIOS_DESLOCADO,TOTAL_COMENTARIOS_DIA_ATUAL,TOTAL_LIKES_DESLOCADO,TOTAL_LIKES_DIA_ATUAL
0,2023-10-15 00:00:00,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,4279413,5415.0,78174.0,0,4279413,0,5415.0,0,78174.0
231,2023-10-16 00:00:00,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,4695234,5706.0,83626.0,4279413.0,415821.0,5415.0,291.0,78174.0,5452.0
471,2023-10-19 00:00:00,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,5407456,6045.0,89762.0,4695234.0,712222.0,5706.0,339.0,83626.0,6136.0
1,2023-10-15 00:00:00,1,CYcrmsdZuyw,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,4141018,17758.0,171952.0,0,4141018,0,17758.0,0,171952.0
232,2023-10-16 00:00:00,1,CYcrmsdZuyw,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,4355662,18443.0,181368.0,4141018.0,214644.0,17758.0,685.0,171952.0,9416.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,2023-10-15 00:00:00,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,145976,236.0,2956.0,0,145976,0,236.0,0,2956.0
470,2023-10-16 00:00:00,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,147478,233.0,2970.0,145976.0,1502.0,236.0,-3.0,2956.0,14.0
700,2023-10-19 00:00:00,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,151752,232.0,2984.0,147478.0,4274.0,233.0,-1.0,2970.0,14.0
921,2023-10-20 00:00:00,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,153609,232.0,2985.0,151752.0,1857.0,232.0,0.0,2984.0,1.0


,DATA_EXTRACAO,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES_DIA_ATUAL,TOTAL_COMENTARIOS_DIA_ATUAL,TOTAL_LIKES_DIA_ATUAL
0,2023-10-15 00:00:00,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,4279413,5415.0,78174.0
231,2023-10-16 00:00:00,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,415821.0,291.0,5452.0
471,2023-10-19 00:00:00,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,712222.0,339.0,6136.0
1,2023-10-15 00:00:00,1,CYcrmsdZuyw,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,4141018,17758.0,171952.0
232,2023-10-16 00:00:00,1,CYcrmsdZuyw,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,214644.0,685.0,9416.0
...,...,...,...,...,...,...,...
230,2023-10-15 00:00:00,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,145976,236.0,2956.0
470,2023-10-16 00:00:00,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,1502.0,-3.0,14.0
700,2023-10-19 00:00:00,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,4274.0,-1.0,14.0
921,2023-10-20 00:00:00,28,v58gkYSVZR8,Eclipse solar: saiba o horário do evento nas c...,1857.0,0.0,1.0


In [21]:
base_max.head(10)

,DATA_EXTRACAO,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES_DIA_ATUAL,TOTAL_COMENTARIOS_DIA_ATUAL,TOTAL_LIKES_DIA_ATUAL
0,2023-10-15 00:00:00,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,4279413,5415.0,78174.0
231,2023-10-16 00:00:00,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,415821.0,291.0,5452.0
471,2023-10-19 00:00:00,1,8KVsaoveTbw,The Iron Claw | Official Trailer HD | A24,712222.0,339.0,6136.0
1,2023-10-15 00:00:00,1,CYcrmsdZuyw,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,4141018,17758.0,171952.0
232,2023-10-16 00:00:00,1,CYcrmsdZuyw,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,214644.0,685.0,9416.0
472,2023-10-19 00:00:00,1,CYcrmsdZuyw,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,528646.0,1004.0,18951.0
701,2023-10-20 00:00:00,1,CYcrmsdZuyw,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,100918.0,295.0,3290.0
922,2023-10-21 00:00:00,1,CYcrmsdZuyw,“Dragon Ball DAIMA” Teaser Trailer / Fall 2024,95882.0,158.0,2492.0
2,2023-10-15 00:00:00,1,KKB5C1Dun-Q,INCRÍVEL PISCINA de BOLINHAS DENTRO do CARRO,269891,114.0,3684.0
233,2023-10-16 00:00:00,1,KKB5C1Dun-Q,INCRÍVEL PISCINA de BOLINHAS DENTRO do CARRO,1771.0,1.0,19.0


In [22]:
base_soma = base_max.groupby(['DATA_EXTRACAO', 'ID_CATEGORIA']).agg(
    TOTAL_VISUALIZACOES=('TOTAL_VISUALIZACOES_DIA_ATUAL', 'sum'),
    TOTAL_COMENTARIOS=('TOTAL_COMENTARIOS_DIA_ATUAL', 'sum'),
    TOTAL_LIKES=('TOTAL_LIKES_DIA_ATUAL', 'sum'),
).reset_index()

base_soma[['TOTAL_VISUALIZACOES', 'TOTAL_COMENTARIOS', 'TOTAL_LIKES']] = base_soma[['TOTAL_VISUALIZACOES', 'TOTAL_COMENTARIOS', 'TOTAL_LIKES']].astype('int')
base_soma.sort_values(by=['TOTAL_VISUALIZACOES'], ascending=False)

,DATA_EXTRACAO,ID_CATEGORIA,TOTAL_VISUALIZACOES,TOTAL_COMENTARIOS,TOTAL_LIKES
6,2023-10-15,24,285267739,572539,11743140
1,2023-10-15,10,188470119,505272,8054410
31,2023-10-19,24,51989464,100616,2219157
44,2023-10-20,24,30917825,133974,882030
3,2023-10-15,20,29314189,116548,1747021
...,...,...,...,...,...
40,2023-10-20,2,6535,58,423
78,2023-10-23,2,6156,19,366
33,2023-10-19,26,2742,10,224
20,2023-10-16,26,2227,10,199


In [23]:
dataframe = pd.merge(
   lista_categoria, base_soma, how='inner', left_on=['ID'], right_on=['ID_CATEGORIA'])

In [24]:
dataframe = dataframe.sort_values(by=['DATA_EXTRACAO'])

In [25]:
dataframe.columns

Index(['ID', 'NOME_CATEGORIA', 'DATA_EXTRACAO', 'ID_CATEGORIA',
       'TOTAL_VISUALIZACOES', 'TOTAL_COMENTARIOS', 'TOTAL_LIKES'],
      dtype='object')

In [26]:
dataframe

,ID,NOME_CATEGORIA,DATA_EXTRACAO,ID_CATEGORIA,TOTAL_VISUALIZACOES,TOTAL_COMENTARIOS,TOTAL_LIKES
0,1,Film & Animation,2023-10-15,1,10646952,36792,487262
102,26,Howto & Style,2023-10-15,26,49953,690,9076
91,25,News & Politics,2023-10-15,25,11520764,13235,248596
80,24,Entertainment,2023-10-15,24,285267739,572539,11743140
69,23,Comedy,2023-10-15,23,1361139,1486,68949
...,...,...,...,...,...,...,...
79,23,Comedy,2023-10-27,23,93108,108,3210
90,24,Entertainment,2023-10-27,24,27166750,74545,1231578
101,25,News & Politics,2023-10-27,25,115333,831,4246
46,19,Travel & Events,2023-10-27,19,46462,280,3604


In [27]:
dataframe.query('ID_CATEGORIA == "20"')


,ID,NOME_CATEGORIA,DATA_EXTRACAO,ID_CATEGORIA,TOTAL_VISUALIZACOES,TOTAL_COMENTARIOS,TOTAL_LIKES
47,20,Gaming,2023-10-15,20,29314189,116548,1747021
48,20,Gaming,2023-10-16,20,8515606,27967,495852
49,20,Gaming,2023-10-19,20,8804495,18400,430599
50,20,Gaming,2023-10-20,20,2081774,3743,90364
51,20,Gaming,2023-10-21,20,4727055,7296,257695
52,20,Gaming,2023-10-22,20,6647347,33157,401200
53,20,Gaming,2023-10-23,20,8572491,34914,706632
54,20,Gaming,2023-10-24,20,9262972,19823,419687
55,20,Gaming,2023-10-25,20,4755872,6704,166900
56,20,Gaming,2023-10-26,20,6455298,24524,244644


In [28]:
dataframe.drop(columns=['ID'], axis=1, inplace=True)
dataframe['NOME_CATEGORIA'] = dataframe['NOME_CATEGORIA'].astype('string')

In [29]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135 entries, 0 to 134
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   NOME_CATEGORIA       135 non-null    string        
 1   DATA_EXTRACAO        135 non-null    datetime64[ns]
 2   ID_CATEGORIA         135 non-null    object        
 3   TOTAL_VISUALIZACOES  135 non-null    int32         
 4   TOTAL_COMENTARIOS    135 non-null    int32         
 5   TOTAL_LIKES          135 non-null    int32         
dtypes: datetime64[ns](1), int32(3), object(1), string(1)
memory usage: 5.8+ KB


In [30]:
dataframe.to_parquet('../data/ouro_csv/trends/popularidade_categoria_trends.parquet')

- Vídeo populares por categoria (Top 10) (likes, comentarios e visualizações) Selecionado por dia

In [31]:
base_original.columns

Index(['ID_CATEGORIA', 'ID_CANAL', 'NM_CANAL', 'ID_VIDEO', 'TITULO_VIDEO',
       'DURACAO', 'DESCRICAO', 'TOTAL_COMENTARIOS', 'TOTAL_FAVORITOS',
       'TOTAL_LIKES', 'TOTAL_VISUALIZACOES', 'DATA_EXTRACAO_API',
       'DATA_EXTRACAO', 'HORA_EXTRACAO', 'TURNO_EXTRACAO', 'SEMANA_EXTRACAO',
       'INDICE_TURNO_EXTRACAO', 'INDICE_SEMANA_EXTRACAO'],
      dtype='object')

In [32]:
df_videos_populares_categoria = base_original[
    ['DATA_EXTRACAO', 'ID_CATEGORIA',  'ID_VIDEO', 'TITULO_VIDEO', 'TOTAL_COMENTARIOS', 'TOTAL_FAVORITOS','TOTAL_LIKES', 'TOTAL_VISUALIZACOES']
]
# df_videos_populares_categoria = df_videos_populares_categoria.query('ID_CATEGORIA == "20" and DATA_EXTRACAO == "2023-10-15"')

In [33]:
df_teste = df_videos_populares_categoria.groupby(['DATA_EXTRACAO','ID_CATEGORIA', 'ID_VIDEO', 'TITULO_VIDEO']).agg(
    TOTAL_VISUALIZACOES=('TOTAL_VISUALIZACOES', 'max'),
    TOTAL_COMENTARIOS=('TOTAL_COMENTARIOS', 'max'),
    TOTAL_LIKES=('TOTAL_LIKES', 'max')
).reset_index().sort_values(by= ['ID_VIDEO', 'DATA_EXTRACAO'])
df_teste.head(50)

,DATA_EXTRACAO,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES,TOTAL_COMENTARIOS,TOTAL_LIKES
93,2023-10-15,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,133464,168.0,5392.0
315,2023-10-16,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,161525,174.0,6173.0
545,2023-10-19,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,284886,187.0,8533.0
776,2023-10-20,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,346730,197.0,9790.0
993,2023-10-21,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,452444,209.0,11836.0
1223,2023-10-22,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,544631,235.0,13420.0
606,2023-10-19,24,-BXItmWzWNE,skibidi toilet 66,10476171,44069.0,956913.0
831,2023-10-20,24,-BXItmWzWNE,skibidi toilet 66,29685000,163211.0,1552518.0
1051,2023-10-21,24,-BXItmWzWNE,skibidi toilet 66,39102892,175630.0,1706637.0
1290,2023-10-22,24,-BXItmWzWNE,skibidi toilet 66,44361844,181024.0,1773829.0


In [34]:
df_teste['TOTAL_VISUALIZACOES_DESLOCADO'] = df_teste.groupby(['ID_VIDEO'])['TOTAL_VISUALIZACOES'].shift(1)
df_teste.fillna(0, axis=1, inplace=True)
df_teste['TOTAL_VISUALIZACOES_DIA_ATUAL'] = df_teste['TOTAL_VISUALIZACOES'] - df_teste['TOTAL_VISUALIZACOES_DESLOCADO']

df_teste['TOTAL_COMENTARIOS_DESLOCADO'] = df_teste.groupby(['ID_VIDEO'])['TOTAL_COMENTARIOS'].shift(1)
df_teste.fillna(0, axis=1, inplace=True)
df_teste['TOTAL_COMENTARIOS_DIA_ATUAL'] = df_teste['TOTAL_COMENTARIOS'] - df_teste['TOTAL_COMENTARIOS_DESLOCADO']

df_teste['TOTAL_LIKES_DESLOCADO'] = df_teste.groupby(['ID_VIDEO'])['TOTAL_LIKES'].shift(1)
df_teste.fillna(0, axis=1, inplace=True)
df_teste['TOTAL_LIKES_DIA_ATUAL'] = df_teste['TOTAL_LIKES'] - df_teste['TOTAL_LIKES_DESLOCADO']
df_teste[['DATA_EXTRACAO', 'ID_CATEGORIA', 'ID_VIDEO', 'TOTAL_VISUALIZACOES', 'TOTAL_VISUALIZACOES_DESLOCADO', 'TOTAL_VISUALIZACOES_DIA_ATUAL']]

,DATA_EXTRACAO,ID_CATEGORIA,ID_VIDEO,TOTAL_VISUALIZACOES,TOTAL_VISUALIZACOES_DESLOCADO,TOTAL_VISUALIZACOES_DIA_ATUAL
93,2023-10-15 00:00:00,20,-8A4gKs9CxE,133464,0,133464
315,2023-10-16 00:00:00,20,-8A4gKs9CxE,161525,133464.0,28061.0
545,2023-10-19 00:00:00,20,-8A4gKs9CxE,284886,161525.0,123361.0
776,2023-10-20 00:00:00,20,-8A4gKs9CxE,346730,284886.0,61844.0
993,2023-10-21 00:00:00,20,-8A4gKs9CxE,452444,346730.0,105714.0
...,...,...,...,...,...,...
813,2023-10-20 00:00:00,20,zzoBoifxk48,232928,232429.0,499.0
1032,2023-10-21 00:00:00,20,zzoBoifxk48,233514,232928.0,586.0
1270,2023-10-22 00:00:00,20,zzoBoifxk48,233794,233514.0,280.0
1507,2023-10-23 00:00:00,20,zzoBoifxk48,234182,233794.0,388.0


In [35]:
df_teste.drop(['TOTAL_COMENTARIOS_DESLOCADO', 'TOTAL_COMENTARIOS', 'TOTAL_VISUALIZACOES_DESLOCADO', 'TOTAL_VISUALIZACOES', 'TOTAL_LIKES_DESLOCADO', 'TOTAL_LIKES'], axis=1, inplace=True)
df_teste.head(45)

,DATA_EXTRACAO,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES_DIA_ATUAL,TOTAL_COMENTARIOS_DIA_ATUAL,TOTAL_LIKES_DIA_ATUAL
93,2023-10-15 00:00:00,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,133464,168.0,5392.0
315,2023-10-16 00:00:00,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,28061.0,6.0,781.0
545,2023-10-19 00:00:00,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,123361.0,13.0,2360.0
776,2023-10-20 00:00:00,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,61844.0,10.0,1257.0
993,2023-10-21 00:00:00,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,105714.0,12.0,2046.0
1223,2023-10-22 00:00:00,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,92187.0,26.0,1584.0
606,2023-10-19 00:00:00,24,-BXItmWzWNE,skibidi toilet 66,10476171,44069.0,956913.0
831,2023-10-20 00:00:00,24,-BXItmWzWNE,skibidi toilet 66,19208829.0,119142.0,595605.0
1051,2023-10-21 00:00:00,24,-BXItmWzWNE,skibidi toilet 66,9417892.0,12419.0,154119.0
1290,2023-10-22 00:00:00,24,-BXItmWzWNE,skibidi toilet 66,5258952.0,5394.0,67192.0


In [36]:
df_teste = pd.merge(df_teste, lista_categoria, left_on=['ID_CATEGORIA'], right_on=['ID'], how='inner')
df_teste

,DATA_EXTRACAO,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES_DIA_ATUAL,TOTAL_COMENTARIOS_DIA_ATUAL,TOTAL_LIKES_DIA_ATUAL,ID,NOME_CATEGORIA
0,2023-10-15 00:00:00,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,133464,168.0,5392.0,20,Gaming
1,2023-10-16 00:00:00,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,28061.0,6.0,781.0,20,Gaming
2,2023-10-19 00:00:00,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,123361.0,13.0,2360.0,20,Gaming
3,2023-10-20 00:00:00,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,61844.0,10.0,1257.0,20,Gaming
4,2023-10-21 00:00:00,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,105714.0,12.0,2046.0,20,Gaming
...,...,...,...,...,...,...,...,...,...
2552,2023-10-23 00:00:00,26,ua1DsYjuavo,TOUR NO MEU SÍTIO,38867.0,314.0,2563.0,26,Howto & Style
2553,2023-10-24 00:00:00,26,ua1DsYjuavo,TOUR NO MEU SÍTIO,16162.0,105.0,995.0,26,Howto & Style
2554,2023-10-25 00:00:00,26,ua1DsYjuavo,TOUR NO MEU SÍTIO,26965.0,274.0,2406.0,26,Howto & Style
2555,2023-10-26 00:00:00,26,ua1DsYjuavo,TOUR NO MEU SÍTIO,35736.0,303.0,2814.0,26,Howto & Style


In [37]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   DATA_EXTRACAO                  2557 non-null   object
 1   ID_CATEGORIA                   2557 non-null   object
 2   ID_VIDEO                       2557 non-null   object
 3   TITULO_VIDEO                   2557 non-null   object
 4   TOTAL_VISUALIZACOES_DIA_ATUAL  2557 non-null   object
 5   TOTAL_COMENTARIOS_DIA_ATUAL    2557 non-null   object
 6   TOTAL_LIKES_DIA_ATUAL          2557 non-null   object
 7   ID                             2557 non-null   object
 8   NOME_CATEGORIA                 2557 non-null   object
dtypes: object(9)
memory usage: 179.9+ KB


In [38]:
df_teste.drop('ID', axis=1, inplace=True)

In [39]:
df_teste['DATA_EXTRACAO'] = df_teste['DATA_EXTRACAO'].astype('datetime64[ns]')
df_teste[['ID_VIDEO', 'TITULO_VIDEO']] = df_teste[['ID_VIDEO', 'TITULO_VIDEO']].astype('string')
df_teste[['TOTAL_VISUALIZACOES_DIA_ATUAL', 'TOTAL_COMENTARIOS_DIA_ATUAL', 'TOTAL_LIKES_DIA_ATUAL']] = df_teste[['TOTAL_VISUALIZACOES_DIA_ATUAL', 'TOTAL_COMENTARIOS_DIA_ATUAL', 'TOTAL_LIKES_DIA_ATUAL']].astype('int32')
df_teste['NOME_CATEGORIA'] = df_teste['NOME_CATEGORIA'].astype('string')
df_teste['ID_CATEGORIA'] = df_teste['ID_CATEGORIA'].astype('int32')
df_teste

,DATA_EXTRACAO,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES_DIA_ATUAL,TOTAL_COMENTARIOS_DIA_ATUAL,TOTAL_LIKES_DIA_ATUAL,NOME_CATEGORIA
0,2023-10-15,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,133464,168,5392,Gaming
1,2023-10-16,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,28061,6,781,Gaming
2,2023-10-19,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,123361,13,2360,Gaming
3,2023-10-20,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,61844,10,1257,Gaming
4,2023-10-21,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,105714,12,2046,Gaming
...,...,...,...,...,...,...,...,...
2552,2023-10-23,26,ua1DsYjuavo,TOUR NO MEU SÍTIO,38867,314,2563,Howto & Style
2553,2023-10-24,26,ua1DsYjuavo,TOUR NO MEU SÍTIO,16162,105,995,Howto & Style
2554,2023-10-25,26,ua1DsYjuavo,TOUR NO MEU SÍTIO,26965,274,2406,Howto & Style
2555,2023-10-26,26,ua1DsYjuavo,TOUR NO MEU SÍTIO,35736,303,2814,Howto & Style


In [40]:
df_teste.head()

,DATA_EXTRACAO,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES_DIA_ATUAL,TOTAL_COMENTARIOS_DIA_ATUAL,TOTAL_LIKES_DIA_ATUAL,NOME_CATEGORIA
0,2023-10-15,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,133464,168,5392,Gaming
1,2023-10-16,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,28061,6,781,Gaming
2,2023-10-19,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,123361,13,2360,Gaming
3,2023-10-20,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,61844,10,1257,Gaming
4,2023-10-21,20,-8A4gKs9CxE,ENTREI NO CÉREBRO DO MEU AMIGO!,105714,12,2046,Gaming


In [41]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   DATA_EXTRACAO                  2557 non-null   datetime64[ns]
 1   ID_CATEGORIA                   2557 non-null   int32         
 2   ID_VIDEO                       2557 non-null   string        
 3   TITULO_VIDEO                   2557 non-null   string        
 4   TOTAL_VISUALIZACOES_DIA_ATUAL  2557 non-null   int32         
 5   TOTAL_COMENTARIOS_DIA_ATUAL    2557 non-null   int32         
 6   TOTAL_LIKES_DIA_ATUAL          2557 non-null   int32         
 7   NOME_CATEGORIA                 2557 non-null   string        
dtypes: datetime64[ns](1), int32(4), string(3)
memory usage: 120.0 KB


In [42]:
df_teste.query('ID_CATEGORIA == 20 and DATA_EXTRACAO == "2023-10-15"').sort_values(by=['TOTAL_VISUALIZACOES_DIA_ATUAL'], ascending=False)

,DATA_EXTRACAO,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,TOTAL_VISUALIZACOES_DIA_ATUAL,TOTAL_COMENTARIOS_DIA_ATUAL,TOTAL_LIKES_DIA_ATUAL,NOME_CATEGORIA
254,2023-10-15,20,_9QCfc77eQs,SWORDS REWORK SNEAK PEEK + SHOWCASE | Blox Fruits,2160387,23820,149696,Gaming
94,2023-10-15,20,Gk_S5cDSj40,Goblin Mines District and New MEGA Troop! - Oc...,2002737,2123,41035,Gaming
384,2023-10-15,20,oMtvCkpis_w,He Did the Monster Mash…up! Clash of Clans Mas...,1708271,978,51206,Gaming
479,2023-10-15,20,yxUxNjZuIDA,A NOVA CASA AUTOMÁTICA - Minecraft Em busca da...,1607585,10361,259444,Gaming
12,2023-10-15,20,1BU4VXofbQk,PlayStation 5 - Same Immersive Power. New Slim...,1481250,3777,38623,Gaming
444,2023-10-15,20,vPtJ1-OZzq4,Garten of Banban 6 - Official Teaser Trailer 2,1297883,6207,49778,Gaming
38,2023-10-15,20,8esOoMGciHI,Fui Adotado por SMURF CATS!!!,1245167,1343,46849,Gaming
372,2023-10-15,20,lMuDch0KB40,AS PRIMEIRAS 24H NO SERVIDOR CREATIVE SQUAD 4,1229756,2290,49828,Gaming
26,2023-10-15,20,5m0j3ahWJf4,Evolui meu Pet de um Coelho Fraco Até um Monstro!,1185757,686,30506,Gaming
42,2023-10-15,20,9thpeZq0yoY,"VIREI CAIXA DE SUPERMERCADO DA MADRUGADA, MAS ...",1160242,2418,57641,Gaming


In [43]:
df_teste.to_parquet('../data/ouro_csv/trends/popularidade_video_categoria.parquet')

- Desempenho turno

In [44]:
base_desempenho_turno = base_original[['ID_CATEGORIA', 'ID_CANAL', 'NM_CANAL', 'TOTAL_COMENTARIOS', 'TOTAL_VISUALIZACOES','DATA_EXTRACAO', 'TURNO_EXTRACAO', 'ID_VIDEO', 'TITULO_VIDEO', 'SEMANA_EXTRACAO', 'INDICE_TURNO_EXTRACAO', 'INDICE_SEMANA_EXTRACAO']]

In [45]:
base_original[['ID_CATEGORIA', 'DATA_EXTRACAO', 'TURNO_EXTRACAO','INDICE_TURNO_EXTRACAO',  'ID_VIDEO','TOTAL_VISUALIZACOES']].query('ID_VIDEO == "CYcrmsdZuyw" and DATA_EXTRACAO == "2023-10-16"')

,ID_CATEGORIA,DATA_EXTRACAO,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,ID_VIDEO,TOTAL_VISUALIZACOES
8,1,2023-10-16,Manhã,2,CYcrmsdZuyw,4167503
208,1,2023-10-16,Manhã,2,CYcrmsdZuyw,4167535
408,1,2023-10-16,Manhã,2,CYcrmsdZuyw,4172546
608,1,2023-10-16,Manhã,2,CYcrmsdZuyw,4189265
808,1,2023-10-16,Manhã,2,CYcrmsdZuyw,4198068
1021,1,2023-10-16,Tarde,3,CYcrmsdZuyw,4209495
1229,1,2023-10-16,Tarde,3,CYcrmsdZuyw,4226469
1429,1,2023-10-16,Tarde,3,CYcrmsdZuyw,4226779
1629,1,2023-10-16,Tarde,3,CYcrmsdZuyw,4281975
1829,1,2023-10-16,Tarde,3,CYcrmsdZuyw,4288858


In [46]:
base_desempenho_categoria_turno = base_original.groupby(
    ['ID_CATEGORIA', 'DATA_EXTRACAO', 'TURNO_EXTRACAO','INDICE_TURNO_EXTRACAO',  'ID_VIDEO']
).agg(
    TOTAL_VISUALIZACOES=('TOTAL_VISUALIZACOES', 'max'),
    TOTAL_COMENTARIOS=('TOTAL_COMENTARIOS', 'max'),
    TOTAL_LIKES=('TOTAL_LIKES', 'max')
).reset_index().sort_values(by=['INDICE_TURNO_EXTRACAO'])

base_desempenho_categoria_turno

,ID_CATEGORIA,DATA_EXTRACAO,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,ID_VIDEO,TOTAL_VISUALIZACOES,TOTAL_COMENTARIOS,TOTAL_LIKES
1588,17,2023-10-22,Manhã,2,1u2tH86s3Sw,292827,21.0,14296.0
4837,24,2023-10-22,Manhã,2,Ytfs5zF5iyk,68309,347.0,7154.0
4838,24,2023-10-22,Manhã,2,ZBl48MK17cI,12230172,3760.0,75108.0
4839,24,2023-10-22,Manhã,2,cXGkoPb6LiA,1367265,4249.0,84646.0
4840,24,2023-10-22,Manhã,2,csHayDosvJ4,1320441,1764.0,19474.0
...,...,...,...,...,...,...,...,...
4455,24,2023-10-19,Noite,4,q5TvfefUfdo,75421,563.0,11294.0
4456,24,2023-10-19,Noite,4,qnZ360evVME,233859,268.0,8601.0
4457,24,2023-10-19,Noite,4,qz0yd7XfhfM,177375,579.0,26495.0
4450,24,2023-10-19,Noite,4,lLysob5QGUs,91178,180.0,9481.0


In [47]:
base_desempenho_categoria_turno = base_desempenho_categoria_turno.groupby(['ID_CATEGORIA', 'DATA_EXTRACAO', 'TURNO_EXTRACAO', 'INDICE_TURNO_EXTRACAO']) \
    .agg(
        TOTAL_VISUALIZACOES=('TOTAL_VISUALIZACOES', 'sum')
    ).reset_index()
base_desempenho_categoria_turno['ID_CATEGORIA'] = base_desempenho_categoria_turno['ID_CATEGORIA'].astype('int32')


In [48]:
base_desempenho_categoria_turno.sort_values(by=['ID_CATEGORIA','DATA_EXTRACAO', 'INDICE_TURNO_EXTRACAO'])

,ID_CATEGORIA,DATA_EXTRACAO,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,TOTAL_VISUALIZACOES
1,1,2023-10-15,Tarde,3,10319209
0,1,2023-10-15,Noite,4,10646952
2,1,2023-10-16,Manhã,2,11098135
4,1,2023-10-16,Tarde,3,11106762
3,1,2023-10-16,Noite,4,11356537
...,...,...,...,...,...
369,28,2023-10-26,Tarde,3,2473483
368,28,2023-10-26,Noite,4,2492407
370,28,2023-10-27,Manhã,2,247583
372,28,2023-10-27,Tarde,3,361810


In [49]:
lista_categoria

,ID,NOME_CATEGORIA
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [1597]:
base_desempenho_categoria_turno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID_CATEGORIA           373 non-null    int32         
 1   DATA_EXTRACAO          373 non-null    datetime64[ns]
 2   TURNO_EXTRACAO         373 non-null    string        
 3   INDICE_TURNO_EXTRACAO  373 non-null    int32         
 4   TOTAL_VISUALIZACOES    373 non-null    int32         
dtypes: datetime64[ns](1), int32(3), string(1)
memory usage: 10.3 KB


In [53]:
base_desempenho_categoria_turno

,ID_CATEGORIA,DATA_EXTRACAO,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,TOTAL_VISUALIZACOES
0,1,2023-10-15,Noite,4,10646952
1,1,2023-10-15,Tarde,3,10319209
2,1,2023-10-16,Manhã,2,11098135
3,1,2023-10-16,Noite,4,11356537
4,1,2023-10-16,Tarde,3,11106762
...,...,...,...,...,...
368,28,2023-10-26,Noite,4,2492407
369,28,2023-10-26,Tarde,3,2473483
370,28,2023-10-27,Manhã,2,247583
371,28,2023-10-27,Noite,4,369055


In [56]:
base_desempenho_categoria_turno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID_CATEGORIA           373 non-null    int32         
 1   DATA_EXTRACAO          373 non-null    datetime64[ns]
 2   TURNO_EXTRACAO         373 non-null    string        
 3   INDICE_TURNO_EXTRACAO  373 non-null    int32         
 4   TOTAL_VISUALIZACOES    373 non-null    int32         
dtypes: datetime64[ns](1), int32(3), string(1)
memory usage: 10.3 KB


In [58]:
lista_categoria['ID'] = lista_categoria['ID'].astype('int32')
lista_categoria['NOME_CATEGORIA'] = lista_categoria['NOME_CATEGORIA'].astype('string')
lista_categoria.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              31 non-null     int32 
 1   NOME_CATEGORIA  31 non-null     string
dtypes: int32(1), string(1)
memory usage: 500.0 bytes


In [60]:
base_desempenho_categoria_turno = pd.merge(
    base_desempenho_categoria_turno, 
    lista_categoria, 
    left_on=['ID_CATEGORIA'], 
    right_on=['ID'], 
    how='inner'
)
teste

,ID_CATEGORIA,DATA_EXTRACAO,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO,TOTAL_VISUALIZACOES,ID,NOME_CATEGORIA
0,1,2023-10-15,Noite,4,10646952,1,Film & Animation
1,1,2023-10-15,Tarde,3,10319209,1,Film & Animation
2,1,2023-10-16,Manhã,2,11098135,1,Film & Animation
3,1,2023-10-16,Noite,4,11356537,1,Film & Animation
4,1,2023-10-16,Tarde,3,11106762,1,Film & Animation
...,...,...,...,...,...,...,...
368,28,2023-10-26,Noite,4,2492407,28,Science & Technology
369,28,2023-10-26,Tarde,3,2473483,28,Science & Technology
370,28,2023-10-27,Manhã,2,247583,28,Science & Technology
371,28,2023-10-27,Noite,4,369055,28,Science & Technology


In [61]:
base_desempenho_categoria_turno.to_parquet('../data/ouro_csv/trends/desempenho_categoria_por_turno.parquet')